In [3]:
import dask
import dask.dataframe as df
import numpy as np
import pandas as pd
from scipy import sparse as sp
from math import log
from collections import Counter
import functools 
import sklearn.metrics as metrics
from itertools import combinations, product
from string import ascii_lowercase

In [61]:
def generate_random_string(length):
    return ''.join(np.random.choice(list(ascii_lowercase), size=length))

def generate_key_chunk(params: dict):
    if params['type'] is 'SocialSecurityNumber':
        intervals = np.linspace(1, 100 - 1, num=params['total_chunks']+1, dtype=int)
        data = np.linspace(intervals[params['chunk_pos']], intervals[params['chunk_pos']+1]-1, num=params['chunk_size'], dtype=int)
        data = np.random.permutation(data)
        data = [str(i).zfill(9) for i in data]
        data = list(map(lambda i: '{}-{}-{}'.format(i[:3], i[3:5], i[5:]), data))
    elif params['type'] is 'String':
        length = np.random.randint(params['min'], params['max'])
        vectorized = np.vectorize(lambda x: '{}{}'.format(generate_random_string(length), x))
        data = vectorized(np.arange(params['chunk_size']*params['chunk_pos'],params['chunk_size']*(1+params['chunk_pos'])))
        data = np.random.permutation(data)
    elif params['type'] is 'Integer':
        intervals = np.linspace(params['min'], params['max'], num=params['total_chunks'] + 1, dtype=int)
        data = np.random.randint(intervals[params['chunk_pos']], intervals[params['chunk_pos']+1]-1, params['chunk_size'])
        data = np.random.permutation(data)
    elif params['type'] is 'Float':
        intervals = np.linspace(params['min'], params['max'], num=params['total_chunks'] + 1, dtype=int)
        range = intervals[params['chunk_pos'] + 1] - 1 - intervals[params['chunk_pos']]
        data = intervals[params['chunk_pos']] + np.random.sample(params['chunk_size']) * range
        data = np.random.permutation(data)
    else:
        data = None
    return data

In [63]:
parametros = {}
parametros['type'] = 'Float'
parametros['chunk_size'] = 5
parametros['total_chunks'] = 10
parametros['min'] = 1
parametros['max'] = 100
#parametros['chunk_pos'] = 2 
for i in np.arange(parametros['total_chunks']-1):
    parametros['chunk_pos'] = i
    values = generate_key_chunk(parametros)
    print(values)

[6.53744358 6.71369734 5.373277   8.44746327 3.5012268 ]
[15.51483987 16.54633995 17.37912269 11.50983357 18.33387992]
[20.68936001 25.80055178 25.91642913 26.17872838 22.50027394]
[37.56987429 35.83783748 32.91388414 36.83023431 33.68182836]
[45.56794357 40.71921355 40.3204783  47.42226992 40.97254688]
[51.13994203 52.65091423 57.50709171 58.85285757 52.78084965]
[67.98380339 63.36848163 60.63883066 66.09122452 66.82272295]
[70.7980997  70.76811999 78.39623909 74.18458818 75.92325711]
[82.72470253 86.11613707 84.24494217 85.24239939 81.71875638]


In [36]:
np.linspace(1, 100 - 1, num=11, dtype=int)

array([ 1, 10, 20, 30, 40, 50, 59, 69, 79, 89, 99])

In [43]:
len(list(combinations(ascii_lowercase, 4)))

14950